In [29]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import os
from dotenv import load_dotenv

from crewai import Agent, Task, Crew, Process
from crewai.tools import tool
from google.oauth2 import service_account
from googleapiclient.discovery import build
from datetime import datetime, timedelta
import pytz

In [31]:
import os
from dotenv import load_dotenv

from crewai import Agent, Task, Crew, Process
from crewai.tools import tool
from google.oauth2 import service_account
from googleapiclient.discovery import build
from datetime import datetime, timedelta
import pytz

# Google Calendar API Setup
load_dotenv()

SERVICE_ACCOUNT_FILE = os.getenv("SERVICE_ACCOUNT_FILE")
CALENDAR_ID = os.getenv("EMAIL_ADDRESS")
SCOPES = ['https://www.googleapis.com/auth/calendar.readonly']

credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
service = build('calendar', 'v3', credentials=credentials)

@tool
def check_free_times(timezone='America/Los_Angeles'):
    """Returns free time slots from Google Calendar for the next 72 hours."""
    local_tz = pytz.timezone(timezone)
    now = datetime.now(local_tz)
    end = now + timedelta(hours=72)

    start_utc = now.astimezone(pytz.utc)
    end_utc = end.astimezone(pytz.utc)

    body = {"timeMin": start_utc.isoformat(), "timeMax": end_utc.isoformat(), "items": [{"id": CALENDAR_ID}]}
    events_result = service.freebusy().query(body=body).execute()
    busy_slots = events_result['calendars'][CALENDAR_ID]['busy']
    
    free_slots = []
    prev_end = start_utc
    for slot in busy_slots:
        busy_start = datetime.fromisoformat(slot['start']).astimezone(local_tz)
        if prev_end.astimezone(local_tz) < busy_start:
            free_slots.append(f"Available: {prev_end.astimezone(local_tz)} to {busy_start}")
        prev_end = datetime.fromisoformat(slot['end']).astimezone(local_tz)

    return free_slots if free_slots else "No free slots today."

In [32]:
import undetected_chromedriver as uc
from bs4 import BeautifulSoup
import pandas as pd
import time

@tool
def get_surf_conditions():
    """Scrapes Surfline for surf conditions at my favorite spots."""
    
    urls = {
        "Linda Mar": "https://www.surfline.com/surf-report/linda-mar/5842041f4e65fad6a7708976?view=table",
        "Linda Mar North": "https://www.surfline.com/surf-report/linda-mar-north/5cbf8d85e7b15800014909e8?view=table",
        "North Ocean Beach": "https://www.surfline.com/surf-report/north-ocean-beach/5d9b68deab58860001c7359e?view=table"
    }

    data = []
    reports = {}

    browser = uc.Chrome()

    for spot, url in urls.items():
        browser.get(url)
        time.sleep(3)  # Allow page to load

        for _ in range(2):
            browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)

        soup = BeautifulSoup(browser.page_source, "html.parser")

        elements = soup.find_all('p', class_=['MuiTypography-root', 'MuiTypography-subtitle1', 'mui-style-wx7j7h'])
        today_report = [el.get_text(separator=' ', strip=True) for el in elements if not el.find_all('span', class_=True)]
        reports[spot] = today_report[5:7]

        sizes = [s.get_text(strip=True) for s in soup.find_all('h4', class_='MuiTypography-root MuiTypography-headline SurfCell_height__1K1mF mui-style-1sd06dx')]
        times = [s.get_text(strip=True) for s in soup.find_all('span', class_="MuiTypography-root MuiTypography-overline2 mui-style-1lv7lt2")]
        grades = [s.get_text(strip=True) for s in soup.find_all('span', class_="MuiTypography-root MuiTypography-overline2 RatingsCell_ratingText__sbkig mui-style-2jthhe")]

        base_date = datetime.today()
        formatted_times = [(base_date + timedelta(days=i // 3)).strftime("%B %d, %Y") + " " + t for i, t in enumerate(times)]

        for size, time_value, grade in zip(sizes, formatted_times, grades):
            data.append({
                "spot": spot,
                "size": size,
                "time": time_value,
                "grade": grade
            })

    browser.quit()

    df = pd.DataFrame(data)
    return reports, df if not df.empty else "No surf data available."

In [33]:
# Define Agents
schedule_manager = Agent(
    role="Schedule Manager",
    goal="Find the best time slots when the user is free on {day}. "
        "If the day is today, only find free times starting now.",
    backstory="A highly organized assistant who manages calendars efficiently.",
    tools=[check_free_times],
    verbose=True
)

surf_analyst = Agent(
    role="Surf Analyst",
    goal="Retrieve surf conditions from Surfline's North Ocean Beach, "
        "Linda Mar North, and Linda Mar pages"
        "and determine the best times to surf on {day}.",
    backstory="An expert in surf forecasting who knows the perfect wave conditions.",
    tools=[get_surf_conditions],
    verbose=True
)

surf_coach = Agent(
    role="Surf Coach",
    goal="Based on the surf conditions reported by surf analyst and the "
        "user's surf preferences, decide what are the optimal times to surf on {day}. "
        "you can assume surf conditions stay consistent between the times in the report.",
    backstory="User's personal surf coach. You know that the user is "
        "an intermediate surfer who can enjoy any surf condition that is"
        "poor to fair or better and they like to surf waves over 1ft and "
        "waves under 6ft.",
    tools=[], 
    verbose=True
)

decision_agent = Agent(
    role="Surf Decision Maker",
    goal="Recommend the best surf time based on free slots and surf conditions for {day}."
        "ONLY suggest times that match availability and consider travel.",
    backstory="A strategic planner who loves maximizing good waves with free time. "
              "You should consider travel times: North Ocean Beach (30 min), "
              "Linda Mar North (60 min), and Linda Mar (60 min). "
              "The minimum enjoyable surf session is 45 min. "
              "Never suggest a time that conflicts with a meeting.",
    tools=[],
    verbose=True
)

In [34]:
# Define Tasks
check_schedule_task = Task(
    description="Retrieve the next 3 day's free time slots from Google Calendar.",
    expected_output="List of available time slots for surfing.",
    agent=schedule_manager
)

check_surf_task = Task(
    description="Scrape Surfline for surf conditions",
    expected_output="Surf forecast for the next 3 days including times, wave sizes, and conditions.",
    agent=surf_analyst
)

analyze_surf_preferences_task = Task(
    description="Evaluate surf conditions for {day} against the user's preferences "
            "(wave height between 1ft-6ft, at least 'poor to fair' conditions).",
    expected_output="List of surfable time slots that match the user's preferences.",
    agent=surf_coach
)

decision_task = Task(
    description="Compare free time slots with the user's preferred surf times and make a final recommendation. "
                "DO NOT recommend surf times that conflict with busy calendar slots. "
                "Make sure to account for travel time before suggesting a departure time.",
    expected_output="The best time to surf {day} based on conditions, availability, and travel time. "
                    "If there isn't a good time, say so.",
    agent=decision_agent
)

In [35]:
os.environ["OPENAI_API_KEY"] = os.environ.get("OPENAI_API_KEY")

# Create Crew
surf_crew = Crew(
    agents=[schedule_manager, surf_analyst, surf_coach, decision_agent],
    tasks=[check_schedule_task, check_surf_task, analyze_surf_preferences_task, decision_task],
    process=Process.sequential,
    memory=False
)

# Run Crew for a Specific Day
day_to_check = "March 19-21, 2025"
result = surf_crew.kickoff(inputs={'day': day_to_check})

print(result)

# Agent: Schedule Manager
## Task: Retrieve the next 3 day's free time slots from Google Calendar.


# Agent: Schedule Manager
## Using tool: check_free_times
## Tool Input: 
"{}"
## Tool Output: 
['Available: 2025-03-20 16:17:29.762330-07:00 to 2025-03-20 17:30:00-07:00', 'Available: 2025-03-20 20:00:00-07:00 to 2025-03-21 08:30:00-07:00']


# Agent: Schedule Manager
## Final Answer: 
- March 20, 2025: 
  - Available: 4:17 PM to 5:30 PM 
  - Available: 8:00 PM to 8:30 AM (March 21)


# Agent: Surf Analyst
## Task: Scrape Surfline for surf conditions


KeyboardInterrupt: 